In [15]:
import tensorflow as tf

In [2]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")

f = x*x*y+y+2

In [5]:
init = tf.global_variables_initializer()

In [6]:
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [7]:
result

42

In [8]:
sess = tf.InteractiveSession()

In [9]:
init.run()

In [10]:
result = f.eval()

In [11]:
result

42

In [12]:
f.eval()

42

In [13]:
sess.close()

In [14]:
f.graph

In [15]:
f.graph is tf.get_default_graph()

True

In [16]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph

In [17]:
graph

In [18]:
tf.get_default_graph()

In [20]:
w = tf.constant(3)
v = tf.Variable(w + 2)

with tf.Session() as sess:
    v.initializer.run()
    print(v.eval())

5


In [2]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [3]:
housing = fetch_california_housing()

In [6]:
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': 'California housing dataset.\n\nThe original database is available from StatLib\n\n 

In [7]:
housing.target

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [8]:
housing.target.shape

(20640,)

In [9]:
housing.data

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

In [46]:
tf.reset_default_graph()
housing = fetch_california_housing()

In [47]:
m,n=housing.data.shape

In [48]:
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

In [49]:
housing_data_plus_bias.shape

(20640, 9)

In [50]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")



In [51]:
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [52]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [53]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

In [22]:
# Gradient Descent

In [59]:
n_epochs = 10000
learning_rate = 0.01
tf.reset_default_graph()

In [62]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [64]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred=tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name="mse")
gradients=2/m*tf.matmul(tf.transpose(X),error)
training_op=tf.assign(theta,theta-learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range (n_epochs):
        #if epoch % 100 == 0:
        #    print("Epoch",epoch,"MSE =",mse.eval())
        sess.run(training_op)
            
    best_theta = theta.eval()
    
best_theta
    
    

array([[ 2.0685523 ],
       [ 0.82960993],
       [ 0.11875149],
       [-0.26550868],
       [ 0.30568027],
       [-0.0045031 ],
       [-0.03932596],
       [-0.8998961 ],
       [-0.870549  ]], dtype=float32)

In [ ]:
# Using autodiff

In [65]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred=tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name="mse")
gradients=tf.gradients(mse, [theta])[0]
training_op=tf.assign(theta,theta-learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range (n_epochs):
        #if epoch % 100 == 0:
        #    print("Epoch",epoch,"MSE =",mse.eval())
        sess.run(training_op)
            
    best_theta = theta.eval()
    
best_theta

array([[ 2.0685525 ],
       [ 0.829638  ],
       [ 0.11875452],
       [-0.2655624 ],
       [ 0.30572549],
       [-0.00450217],
       [-0.03932697],
       [-0.8998485 ],
       [-0.87050605]], dtype=float32)

In [66]:
# Using an optimizer

In [69]:
tf.reset_default_graph()
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred=tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range (n_epochs):
        #if epoch % 100 == 0:
        #    print("Epoch",epoch,"MSE =",mse.eval())
        sess.run(training_op)
            
    best_theta = theta.eval()
    
best_theta

array([[ 2.0685525 ],
       [ 0.8296418 ],
       [ 0.11875557],
       [-0.26556912],
       [ 0.30573088],
       [-0.00450179],
       [-0.03932714],
       [-0.89983636],
       [-0.8704949 ]], dtype=float32)